### 1. 安装HuggingFace 并下载模型到本地

In [4]:
!pip install huggingface-hub -Uqq -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install modelscope -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install -U sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.6/896.6 kB 4.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.7 MB/s eta 0:00:0000:01
  Created wheel for sagemaker: filename=sagemaker-2.191.0-py2.py3-none-any.whl size=1197714 sha256=196acff84df6b3641b89640a11c65eac3a93cbecdf2ee229e49b804cd37fcae9
  Stored in directory: /root/.cache/pip/wheels/44/fa/ba/c5c651d1e70cb679f61a537dc1f4b6f5c41f02cad7cf51a879
Successfully built sagemaker
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.184.0
    Uninstalling sagemaker-2.184.0:
      Successfully uninstalled sagemaker-2.184.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are

In [6]:
!rm -rf ./LLM_qwen_int4_model

In [6]:
from huggingface_hub import snapshot_download
from pathlib import Path
local_model_path = Path("./LLM_qwen_int4_model")
local_model_path.mkdir(exist_ok=True)

In [7]:
# model_name = "Qwen/Qwen-7B-Chat-Int4"
# commit_hash = "b725fe596dce755fe717c5b15e5c8243d5474f66"

In [8]:
# snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

'LLM_qwen_int4_model/models--Qwen--Qwen-7B-Chat-Int4/snapshots/b725fe596dce755fe717c5b15e5c8243d5474f66'

## 如果是中国区，从modelscope下载比较快

In [8]:

from modelscope.hub.snapshot_download import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_qwen_int4_model")

local_model_path.mkdir(exist_ok=True)
model_name = "qwen/Qwen-7B-Chat-Int4"
commit_hash = "v1.1.4"

snapshot_download(model_name, revision=commit_hash, cache_dir=local_model_path)


2023-10-08 08:30:40,070 - modelscope - INFO - Use user-specified model revision: v1.1.4
Downloading: 100%|██████████| 8.21k/8.21k [00:00<00:00, 14.0MB/s]
Downloading: 100%|██████████| 50.8k/50.8k [00:00<00:00, 756kB/s]
Downloading: 100%|██████████| 1.17k/1.17k [00:00<00:00, 6.71MB/s]
Downloading: 100%|██████████| 2.29k/2.29k [00:00<00:00, 12.9MB/s]
Downloading: 100%|██████████| 1.88k/1.88k [00:00<00:00, 10.6MB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 1.49MB/s]
Downloading: 100%|██████████| 6.73k/6.73k [00:00<00:00, 1.64MB/s]
Downloading: 100%|█████████▉| 1.90G/1.90G [00:25<00:00, 79.3MB/s]
Downloading: 100%|█████████▉| 1.91G/1.91G [00:26<00:00, 78.3MB/s]
Downloading: 100%|█████████▉| 1.65G/1.65G [00:21<00:00, 83.4MB/s]
Downloading: 100%|██████████| 64.2k/64.2k [00:00<00:00, 988kB/s]
Downloading: 100%|██████████| 55.6k/55.6k [00:00<00:00, 1.10MB/s]
Downloading: 100%|██████████| 2.64k/2.64k [00:00<00:00, 13.3MB/s]
Downloading: 100%|██████████| 214/214 [00:00<00:00, 635kB/s]

'LLM_qwen_int4_model/qwen/Qwen-7B-Chat-Int4'

### 2. 把模型拷贝到S3为后续部署做准备

In [9]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [11]:
s3_model_prefix = "LLM-RAG/workshop/LLM_qwen_int4_stream_model"  # folder where model checkpoint will go
# model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
model_snapshot_path = list(local_model_path.glob("**/qwen/*"))[0]

s3_code_prefix = "LLM-RAG/workshop/LLM_qwen_int4_stream_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_qwen_int4_stream_deploy_code
model_snapshot_path: LLM_qwen_int4_model/qwen/Qwen-7B-Chat-Int4


In [12]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_qwen_int4_model/qwen/Qwen-7B-Chat-Int4/NOTICE to s3://sagemaker-cn-northwest-1-370400458231/LLM-RAG/workshop/LLM_qwen_int4_stream_model/NOTICE
upload: LLM_qwen_int4_model/qwen/Qwen-7B-Chat-Int4/LICENSE to s3://sagemaker-cn-northwest-1-370400458231/LLM-RAG/workshop/LLM_qwen_int4_stream_model/LICENSE
upload: LLM_qwen_int4_model/qwen/Qwen-7B-Chat-Int4/.mdl to s3://sagemaker-cn-northwest-1-370400458231/LLM-RAG/workshop/LLM_qwen_int4_stream_model/.mdl
upload: LLM_qwen_int4_model/qwen/Qwen-7B-Chat-Int4/README.md to s3://sagemaker-cn-northwest-1-370400458231/LLM-RAG/workshop/LLM_qwen_int4_stream_model/README.md
upload: LLM_qwen_int4_model/qwen/Qwen-7B-Chat-Int4/cpp_kernels.py to s3://sagemaker-cn-northwest-1-370400458231/LLM-RAG/workshop/LLM_qwen_int4_stream_model/cpp_kernels.py
upload: LLM_qwen_int4_model/qwen/Qwen-7B-Chat-Int4/.msc to s3://sagemaker-cn-northwest-1-370400458231/LLM-RAG/workshop/LLM_qwen_int4_stream_model/.msc
upload: LLM_qwen_int4_model/qwen/Qwen-7B-Chat-Int4/gen

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [14]:
# 中国区需要替换为下面的image_uri
inference_image_uri = (
    f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

# inference_image_uri = image_uris.retrieve(
#     framework="djl-deepspeed",
#     region=sess.boto_session.region_name,
#     version="0.23.0"
# )
# print(f"Image going to be used is ---- > {inference_image_uri}")

In [15]:
!mkdir -p LLM_qwen_int4_stream_deploy_code

In [16]:
%%writefile LLM_qwen_int4_stream_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers.generation import GenerationConfig
# from auto_gptq import AutoGPTQForCausalLM


STOP_flag = "[DONE]"


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_location, device_map="auto", trust_remote_code=True).eval()
    model.generation_config  = GenerationConfig.from_pretrained(model_location, trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参
    return model, tokenizer, model.generation_config 


model = None
tokenizer = None
generator = None
config = None

def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer, config
    size = 0
    response = ""
    config.max_new_tokens = max_length
    config.top_p = top_p
    
    ##传入temperature会报错
    ##model.generation_config.temperature = temperature 
    res_generator = model.chat_stream(tokenizer, prompt, history=history,generation_config=config)
    for response in res_generator:
        this_response = response[size:]
        size = len(response)
        stream_buffer = { "outputs":this_response,"finished": False}
        yield stream_buffer
    ## stop
    # yield {"query": prompt, "outputs": STOP_flag, "response": response, "history": [], "finished": True}


def handle(inputs: Input):
    global model, tokenizer,config
    if not model:
        model, tokenizer,config = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data.get("history",[])
    stream = data.get('stream',False)
    print(f'input prompt:{input_sentences}')   
    outputs = Output()
    if stream:
        outputs.add_property("content-type", "application/jsonlines")
        outputs.add_stream_content(stream_items(input_sentences,history=history,**params))
    else:
        config.max_new_tokens = params.get('max_length',1024)
        config.top_p = params.get('top_p',1)
        response, history = model.chat(tokenizer, input_sentences, history=history,generation_config=config)
        result = {"outputs": response, "history" : history}
        outputs.add_as_json(result)
        
    return outputs

Writing LLM_qwen_int4_stream_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [35]:
%%writefile LLM_qwen_int4_stream_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url = s3://sagemaker-cn-northwest-1-370400458231/LLM-RAG/workshop/LLM_qwen_int4_stream_model/

Overwriting LLM_qwen_int4_stream_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示
```
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1
```

In [36]:
%%writefile LLM_qwen_int4_stream_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.32.0
accelerate
tiktoken
einops
scipy
transformers_stream_generator==0.0.4
peft
deepspeed
auto-gptq
optimum

Overwriting LLM_qwen_int4_stream_deploy_code/requirements.txt


In [37]:
# !pip install auto-gptq

In [38]:
!rm model.tar.gz
!cd LLM_qwen_int4_stream_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_qwen_int4_stream_deploy_code

LLM_qwen_int4_stream_deploy_code/
LLM_qwen_int4_stream_deploy_code/model.py
LLM_qwen_int4_stream_deploy_code/requirements.txt
LLM_qwen_int4_stream_deploy_code/serving.properties


In [39]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-cn-northwest-1-370400458231/LLM-RAG/workshop/LLM_qwen_int4_stream_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [40]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"qwen-stream-int4") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

qwen-stream-int4-2023-10-08-09-22-02-560
Image going to be used is ---- > 727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws-cn:sagemaker:cn-northwest-1:370400458231:model/qwen-stream-int4-2023-10-08-09-22-02-560


In [41]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws-cn:sagemaker:cn-northwest-1:370400458231:endpoint-config/qwen-stream-int4-2023-10-08-09-22-02-560-config',
 'ResponseMetadata': {'RequestId': 'bc8744a9-9231-4b63-b693-5f69ed30f9e0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bc8744a9-9231-4b63-b693-5f69ed30f9e0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '136',
   'date': 'Sun, 08 Oct 2023 09:22:04 GMT'},
  'RetryAttempts': 0}}

In [42]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws-cn:sagemaker:cn-northwest-1:370400458231:endpoint/qwen-stream-int4-2023-10-08-09-22-02-560-endpoint


#### 持续检测模型部署进度

In [43]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws-cn:sagemaker:cn-northwest-1:370400458231:endpoint/qwen-stream-int4-2023-10-08-09-22-02-560-endpoint
Status: InService


### 5. 模型测试

In [44]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 1024,
  "temperature": 0.1,
  "top_p":0.8
}

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.89 ms


In [45]:
import io


class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

## Stream

In [46]:
# prompts1 = """你"""
import time

start = time.time()
prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
请问AWS Clean Rooms是多方都会收费吗？
"""
prompts1 = """你好"""
response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : [],
                "stream" : True,
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
scanner = StreamScanner()
for event in event_stream:
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)
            # print(resp)
            print(resp.get("outputs")['outputs'], end='')
        except Exception as e:
            print(line)
            continue
print (f"\ntime:{time.time()-start} s")

你好！有什么我能为你效劳的吗？
time:1.3518805503845215 s


## None stream

In [48]:
import time
parameters = {
  "max_length": 1024,
  "temperature": 0.1,
  "top_p":0.8
}

# prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
# 请问AWS Clean Rooms是多方都会收费吗？
# """

prompts1 = """你现在是口袋奇兵(TopWar:Battle Game)的专属智能客服TopWarBot，你是一个非常专业的游戏客服，，请严格确理解的根据反引号中的资料提取相关信息，回答指挥官的各种问题，不可联想推测反引号中没有的信息,不可提供带有岐义和模棱两可的回答。
```
问题: 钛蓝金币转换成钻石的比例
回答: 按照1：1的比例i进行转换，1个钛蓝金币=1个钻石

问题: 金币
回答: 1、金币的定义：金币是《口袋奇兵》世界中的通用货币
2、金币的获得方法：在游戏内可以通过税收中心/金币收割机进行获取。需要放置金矿生产金币，同时有部分装饰是可以提高金币产量。税收中心只需要20个当前玩家等级民居即可达到之前造满民居的收益。新版本的金币自动收割机是需要10个当前等级的金矿。
3、金币的用途：建筑升级、装备制造与改造等大部分游戏功能均需要消耗一定数量的金币

问题: 远征行动中如何查看小地图与大地图
回答: 在远征行动主页面点击游戏正上方的小地图，可以查看当前关卡的信息，在点击右上角的“地图”就可以观看远征行动大地图信息

问题: 数值单位
回答: 游戏的数值显示单位为
k→m→b→t→aa→bb→cc→dd→ee→ff→gg→hh依次递进，1000进一个单位，比如1000=1k，1000k=1m，1000m=1b，1000b=1t，1000t=1aa，1000aa=1bb，1000bb=1cc。
```

指挥官: 金币单位t大还是b大
TopWarBot: 
"""

prompts2 = """你好"""
start = time.time()

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : [],
            }
            ),
            ContentType="application/json",
        )

print(response_model['Body'].read().decode('utf8'))
print (f"\ntime:{time.time()-start} s")

{
  "outputs":"1000b＞1000t",
  "history":[
    [
      "你现在是口袋奇兵(TopWar:Battle Game)的专属智能客服TopWarBot，你是一个非常专业的游戏客服，，请严格确理解的根据反引号中的资料提取相关信息，回答指挥官的各种问题，不可联想推测反引号中没有的信息,不可提供带有岐义和模棱两可的回答。\n```\n问题: 钛蓝金币转换成钻石的比例\n回答: 按照1：1的比例i进行转换，1个钛蓝金币=1个钻石\n\n问题: 金币\n回答: 1、金币的定义：金币是《口袋奇兵》世界中的通用货币\n2、金币的获得方法：在游戏内可以通过税收中心/金币收割机进行获取。需要放置金矿生产金币，同时有部分装饰是可以提高金币产量。税收中心只需要20个当前玩家等级民居即可达到之前造满民居的收益。新版本的金币自动收割机是需要10个当前等级的金矿。\n3、金币的用途：建筑升级、装备制造与改造等大部分游戏功能均需要消耗一定数量的金币\n\n问题: 远征行动中如何查看小地图与大地图\n回答: 在远征行动主页面点击游戏正上方的小地图，可以查看当前关卡的信息，在点击右上角的“地图”就可以观看远征行动大地图信息\n\n问题: 数值单位\n回答: 游戏的数值显示单位为\nk→m→b→t→aa→bb→cc→dd→ee→ff→gg→hh依次递进，1000进一个单位，比如1000=1k，1000k=1m，1000m=1b，1000b=1t，1000t=1aa，1000aa=1bb，1000bb=1cc。\n```\n\n指挥官: 金币单位t大还是b大\nTopWarBot: \n",
      "1000b＞1000t"
    ]
  ]
}

time:1.2766568660736084 s


#### 清除模型Endpoint和config

In [32]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}


An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "qwen-stream-int4-2023-10-08-08-55-00-416-endpoint".


In [33]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}


An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "qwen-stream-int4-2023-10-08-08-55-00-416-config".


In [34]:
!aws sagemaker delete-model --model-name {model_name}


An error occurred (ValidationException) when calling the DeleteModel operation: Could not find model "qwen-stream-int4-2023-10-08-08-55-00-416".
